# Type Type relation view


In [1]:
from pyld import jsonld
import json
import pyoxigraph
from pyoxigraph import RdfFormat
import io

from pyoxigraph.pyoxigraph import RdfFormat
from rdflib import Graph
import pandas as pd
import networkx as nx
from ipysigma import Sigma


from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML
from hashlib import sha256

In [2]:
store = pyoxigraph.Store()  #    store = pyoxigraph.Store(path="./store")
mime_type = RdfFormat.N3 # "application/n-triples"

In [3]:
# Read sitemap and load JSON-LD content into the store
import requests
import xml.etree.ElementTree as ET
from urllib.parse import urlparse
import os
import io

def read_sitemap(sitemap_path):
    """Read sitemap XML file and extract URLs."""
    # Parse the XML file
    tree = ET.parse(sitemap_path)
    root = tree.getroot()

    # Define the namespace
    ns = {'sm': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

    # Extract URLs from the sitemap
    urls = []
    for url_elem in root.findall('.//sm:url', ns):
        loc_elem = url_elem.find('sm:loc', ns)
        if loc_elem is not None and loc_elem.text:
            urls.append(loc_elem.text)

    return urls

def download_and_process(url, store):
    """Download JSON-LD content from URL and load it into the store."""
    try:
        print(f"Processing: {url}")

        # Download the content
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the JSON-LD content
        c = response.json()

        # Normalize the JSON-LD to N-Quads format
        normalized = jsonld.normalize(c, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})

        # Load the normalized data into the store
        store.load(io.StringIO(normalized), mime_type)

        print(f"Successfully loaded: {url}")
        return True
    except Exception as e:
        print(f"Error processing {url}: {str(e)}")
        return False

# Path to the sitemap file (can be changed to use a different sitemap)
sitemap_path = "../output/jld-sitemap.xml"  # Default sitemap file
# Uncomment the line below to use a different sitemap file
# sitemap_path = "../output/jdlnew-sitemap.xml"

# Extract URLs from the sitemap
urls = read_sitemap(sitemap_path)
print(f"Found {len(urls)} URLs in the sitemap: {sitemap_path}")

# Process each URL and load into the store
successful = 0
for url in urls:
    if download_and_process(url, store):
        successful += 1

print(f"Successfully processed {successful} out of {len(urls)} URLs")

Found 71 URLs in the sitemap: ../output/jld-sitemap.xml
Processing: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_085206_s41586_023_06801_2.json
Successfully loaded: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_085206_s41586_023_06801_2.json
Processing: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_085208_pmc.json
Successfully loaded: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_085208_pmc.json
Processing: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_085217_s41590_024_01915_9.json
Successfully loaded: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_085217_s41590_024_01915_9.json
Processing: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_085221_www.json
Successfully loaded: https://raw.githubusercontent.com/bio-xyz/BioAgents/main/sampleJsonLds/20250214_

In [4]:
# OPTIONAL save as an RDF file
store.dump("../output/biograph.nt", format=RdfFormat.N_QUADS)  #  odd that it wants quads even for a graph without named graphs?

In [5]:
rq1 = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT ?source ?type ?target ?sType ?tType ?sname ?tname
WHERE {
    ?source a ?sType .
    ?target a ?tType .
    ?source ?type ?target .
    OPTIONAL {?source schema:name ?sname} .
    OPTIONAL { ?target schema:name ?tname }.
}
"""

rq4 = """PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT DISTINCT  ?source ?name ?type ?target ?sType ?tType
WHERE {
BIND (schema:Dataset as ?sType) .
BIND (schema:author  as ?type) .

?source a ?sType .
?source schema:citation ?citation .
?citation ?type ?target .
?target schema:name ?name .
?target a ?tType
}
"""


In [6]:
qr = list(store.query(rq1))


In [7]:
print(len(qr))


1061


In [8]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])
g = GraphML()
g.directed = False

for r in qr:
    n1 = g.add_node(r['source'])
    n1['type'] = r['sType']
    if r['sname']:  # Check if sname is valid
        n1['name'] = r['sname']

    n2 = g.add_node(r['target'])
    n2['type'] = r['tType']
    if r['tname']:  # Check if tname is valid
        n2['name'] = r['tname']
    # n2['value'] = row['funding.name']

    # n3 = g.add_node(row['ahash'])
    # n3['type'] = "Affiliation"
    # n3['value'] = row['affil']

    e1 = g.add_edge(n1, n2)
    # e2 = g.add_edge(n1, n3)
    # e['predicate'] = row['type']

In [9]:
fname = "testGraphML.graphml"
parser = GraphMLParser()
parser.write(g, fname)

In [10]:
g = nx.read_graphml("testGraphML.graphml")

In [14]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='type',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)

Sigma(nx.MultiGraph with 471 nodes and 1,061 edges)

In [12]:
from typing import Union, Any

def custom_node_color_mapping(node: dict[str, Any]):
    """let the color be orange or blue if the index is even or odd respectively"""
    return ("#eb4934" if node['properties']['type'] == "Dataset" else "#2456d4")

In [13]:
from yfiles_jupyter_graphs import GraphWidget

w = GraphWidget(graph=g)
w.set_sidebar(enabled=False)
w.set_node_color_mapping(custom_node_color_mapping)

display(w)

GraphWidget(layout=Layout(height='800px', width='100%'))